In [ ]:
from google.colab import files
uploaded = files.upload()

Saving файл 2.csv to файл 2.csv


In [ ]:
import pandas as pd
from pathlib import Path
import os
from sklearn.metrics import classification_report, fbeta_score, accuracy_score

Загрузим файл, переведём его в dataframe и посмотрим на основную информацию о столбцах, также посмортим на саму таблицу

In [ ]:
def get_df(dir):
    df = pd.read_csv(dir, sep='\t', header=0, skipinitialspace=True)
    return df

workdir = Path(os.getcwd())
df = get_df(workdir/'файл 2.csv')
print(df.info())
print(df.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   login   250000 non-null  object
 1   uid     250000 non-null  int64 
 2   docid   250000 non-null  int64 
 3   jud     250000 non-null  int64 
 4   cjud    250000 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 9.5+ MB
None
         login  uid  docid  jud  cjud
0  assessor158  158      0    0     0
1  assessor238  238      0    0     0
2  assessor488  488      0    0     0
3  assessor136  136      0    0     0
4  assessor300  300      0    0     0


Посмотрим на количество оценок каждого асессора. Минимальное количество оценок у ассесора равно 99, значит можно спокойно оценивать работу каждого ассесора 

In [ ]:
print(df['login'].value_counts())

assessor191    484
assessor140    481
assessor161    480
assessor236    467
assessor165    467
              ... 
assessor420    365
assessor202    365
assessor255    364
assessor550    128
assessor234     99
Name: login, Length: 600, dtype: int64


Для оценки работы ассесоров можно применить стандартые метрики классификации: accuracy, precision, recall, f1-score. Для этого напишем функцию агрегации по датафрефрейму сгруппированому по user_id. По итогу получим dataframe со всеми перечисленными метриками для каждого ассесора.

In [ ]:
group_login = df.groupby('uid', as_index=False)

def get_metriс(df):
  clf_report = classification_report(df['cjud'], df['jud'], output_dict=True)
  result = clf_report['1']
  result['accuracy'] = clf_report['accuracy']
  del result['support']
  return pd.Series(result, index=['accuracy', 'precision', 'recall', 'f1-score'])

metric_df = group_login.apply(get_metriс)
print(metric_df)

     uid  accuracy  precision    recall  f1-score
0      0  0.837905   0.453704  0.890909  0.601227
1      1  0.800971   0.344538  0.911111  0.500000
2      2  0.799472   0.403670  0.800000  0.536585
3      3  0.460094   0.108225  0.510204  0.178571
4      4  0.827751   0.452991  0.868852  0.595506
..   ...       ...        ...       ...       ...
595  595  0.849099   0.393617  0.787234  0.524823
596  596  0.917085   0.610390  0.940000  0.740157
597  597  0.892774   0.581081  0.741379  0.651515
598  598  0.775120   0.330645  0.788462  0.465909
599  599  0.905660   0.605263  0.821429  0.696970

[600 rows x 5 columns]


Немного о природе этого бинарного задания. Ассесор будет присваивать 1, если документ не релевантен.

accuracy - это процент верно распознаных классов 1 или 0. Это простая метрика может стать основной для оценки работы ассесоров. Только 25% ассесоров верно распознают класс с вероятностью ниже 0.81. Другие 25% ассеосоров верно распознают класс с вероятностью выше 0.89 (при описании цифры немного округляю). Средняя вероятность правильного распознования класса у ассесора равна 0.84.

In [ ]:
print(metric_df['accuracy'].describe())

count    600.000000
mean       0.840658
std        0.055462
min        0.425791
25%        0.808538
50%        0.833333
75%        0.887500
max        0.923810
Name: accuracy, dtype: float64


Precision можно интерпретировать как долю документов, отнесённых ассесором к классу 1 и при этом действительно являющимися релевантными. Это означает, что нам не так важно найти все нерелевантные документы, как быть уверенным в том, что среди документов названых ассесором нерелевантными большинство окажется нерелевантными. Эта метрика нам точно не подходит, поскольку она не контролирует количество нерелевантных документов ненайденных ассесором. Если ассесор узнает, что его работу оценивают такой метрикой, то он может начать отмечать как нерелевантные документы только те, в которых он хорошо уверен. И если ассесор увидит документ, с которым 'не все так просто', он может просто пропусть его и на оценку это не повлияет.

In [ ]:
print(metric_df['precision'].describe())

count    600.000000
mean       0.431491
std        0.101006
min        0.097458
25%        0.361286
50%        0.411528
75%        0.515396
max        0.666667
Name: precision, dtype: float64


Recall показывает какую долю документов ассесор отметил как нерелевантные из всех реально нерелевантных документов. Эта метрика чуть больше подходит для этой задачи, но все равно плоха. Поскольку нам важно распознать как можно больше нерелеватных документов, но ассесор может начать бездумно отмечать документы как нерелевантные. Это будет неприятно для многих релевантных документов. Собственно говоря, если сравнить precision и recall, то видно, что если ассесор сомневается, то он скорее отнесет документ к нерелеватному. Об этом говорит, то что precision заметно меньше, чем recall.

In [ ]:
print(metric_df['recall'].describe())

count    600.000000
mean       0.830376
std        0.086267
min        0.105263
25%        0.796296
50%        0.845491
75%        0.886364
max        0.982456
Name: recall, dtype: float64


F-мера это среднее гармоническое (с множителем 2, чтобы в случае precision = 1 и recall = 1 иметь F = 1). F-мера достигает максимума при recall и precision, равными единице, и близка к нулю, если один из аргументов близок к нулю. Эта метрика хорошо подходит для данной задачи. Нам становиться важно, чтобы ассесор с одной стороны нашел как можно больше нерелеватных документов, при этом старался походу не 'зацеплять' релеватные документы.

In [ ]:
print(metric_df['f1-score'].describe())

count    600.000000
mean       0.562748
std        0.099385
min        0.103896
25%        0.500000
50%        0.557735
75%        0.641580
max        0.751773
Name: f1-score, dtype: float64


На мой взгляд, для этой задачи recall является чуть важнее, чем precision. Важнее, чтобы реальному пользователю не достался нереватный документ, чем то, что какое-то количество релевантных документов отметят как нерелевантные. Поэтому F-меру можно немного улучшить. Взяв коэфицент beta = 1.5 в формуле F-меры, мы отдадим больший приоритет recall. Итоговая формула F-меры будет иметь вид:

F-мера = 1.(4)*Precision×Recall/(Precision+Recall)

Сделаем функцию по шаблону get_metriс для нахождения F-меры с beta=1.5 для каждого ассесора и посмотрим на статистики.

In [ ]:
group_login = df.groupby('uid', as_index=False)

def get_fbeta(df):
  result = fbeta_score(df['cjud'], df['jud'], beta=1.5)
  return pd.Series(result, index=['fbeta-score'])

fbeta_df = group_login.apply(get_fbeta)
print(fbeta_df['fbeta-score'].describe())

count    600.000000
mean       0.639971
std        0.094543
min        0.104418
25%        0.587668
50%        0.645642
75%        0.710569
max        0.826334
Name: fbeta-score, dtype: float64


Также можно оценивать работу ассесора основываясь не только на его способностях, но и на сложности документа. Все документы оцениваются пять раз.

In [ ]:
print(df['docid'].value_counts())

2047     5
29860    5
17698    5
23841    5
21792    5
        ..
12883    5
14930    5
8785     5
10832    5
0        5
Name: docid, Length: 50000, dtype: int64


Расчитаем accuracy для каждого документа.

In [ ]:
group_docid = df.groupby('docid', as_index=False)

def get_metriс(df):
  result = dict()
  result['accuracy'] = accuracy_score(df['cjud'], df['jud'])
  return pd.Series(result, index=['accuracy'])

doc_metric_df = group_docid.apply(get_metriс)
print(doc_metric_df)

       docid  accuracy
0          0       1.0
1          1       0.8
2          2       0.8
3          3       0.8
4          4       0.8
...      ...       ...
49995  49995       1.0
49996  49996       1.0
49997  49997       0.8
49998  49998       0.8
49999  49999       1.0

[50000 rows x 2 columns]


75% документов имееют accuracy больше 0.8. Это говорит о том, что действительно сложных документов для распознавания ассесорами не так много, чтобы оценивать их работу, принимая сложность документа во внимание.

In [ ]:
print(doc_metric_df['accuracy'].describe())

count    50000.000000
mean         0.841288
std          0.163129
min          0.000000
25%          0.800000
50%          0.800000
75%          1.000000
max          1.000000
Name: accuracy, dtype: float64


Метрики классификации независимо от ассесора и документа, также могут использоваться для оценки работы ассесора путём сравнения их с его индивидуальными метриками

In [ ]:
clf_report = classification_report(df['cjud'], df['jud'], output_dict=True)
result = clf_report['1']
result['accuracy'] = clf_report['accuracy']
del result['support']
total_metrics = pd.Series(result, index=['accuracy', 'precision', 'recall', 'f1-score'])
print(total_metrics)

accuracy     0.841288
precision    0.418545
recall       0.831087
f1-score     0.556720
dtype: float64


Например, можно узнать долю людей с f1-score больше, чем в общем по dataframe

In [ ]:
selected = metric_df.loc[metric_df['f1-score'] > total_metrics['f1-score']]
print(len(selected) / len(metric_df))

0.5066666666666667
